In [1]:
import os
from dotenv import load_dotenv
import openai
openai.api_key=os.getenv("OPENAI_API_KEY")

In [2]:
from openai import ChatCompletion, Completion

In [3]:
from typing import Optional, Callable, List, Dict

In [4]:
from pydantic import create_model
import inspect, json
from inspect import Parameter

In [5]:
# first lets do it how the lecture does it, and then we will expand on this with our own module
def askgpt(user: str, system: str, model: str = "gpt-3.5-turbo", **kwargs):
    # The main thing here is the **kwargs that will be fed into the openai function
    messages = []
    if system: messages.append({'role': 'system', 'content': system})
    messages.append({'role': 'user', 'content': user})
    return ChatCompletion.create(model=model, messages=messages, **kwargs)

In [6]:
# next we will create a helper function that can take any python function and give us a json schema that is required for openai to then call that function
def schema(f):
    kw = {n:(o.annotation, ... if o.default==Parameter.empty else o.default)
          for n,o in inspect.signature(f).parameters.items()}
    s = create_model(f"Input for `{f.__name__}`", **kw).model_json_schema()
    return dict(name=f.__name__, description=f.__doc__, parameters=s)

In [7]:
# our first functions
def sums(a: int, b:int = 1):
    "Adds a + b"
    return a + b

def subtract(a: int, b: int):
    "Subtract a - b"
    return a - b

In [8]:
schema(subtract)

{'name': 'subtract',
 'description': 'Subtract a - b',
 'parameters': {'properties': {'a': {'title': 'A', 'type': 'integer'},
   'b': {'title': 'B', 'type': 'integer'}},
  'required': ['a', 'b'],
  'title': 'Input for `subtract`',
  'type': 'object'}}

In [10]:
c = askgpt(
    user="Use the `sum` function to solve this: What is 10 + 50?",
    system="You must use the provided functions to answer the user questions",
    functions=[schema(sums), schema(subtract)]
)

In [11]:
m = c.choices[0].message

In [12]:
m

<OpenAIObject at 0x110817530> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "sums",
    "arguments": "{\n  \"a\": 10,\n  \"b\": 50\n}"
  }
}

## More general to run any python function

In [13]:
import ast, tokenize
from io import BytesIO

In [20]:
funcs_ok = {'sums', 'python'}
def call_func(c):
    fc = c.choices[0].message.function_call
    if fc.name not in funcs_ok: return print(f'Not allowed: {fc.name}')
    f = globals()[fc.name]
    return f(**json.loads(fc.arguments))

In [21]:
def run(code):
    tree = ast.parse(code)
    last_node = tree.body[-1] if tree.body else None
    
    # If the last node is an expression, modify the AST to capture the result
    if isinstance(last_node, ast.Expr):
        tgts = [ast.Name(id='_result', ctx=ast.Store())]
        assign = ast.Assign(targets=tgts, value=last_node.value)
        tree.body[-1] = ast.fix_missing_locations(assign)

    ns = {}
    exec(compile(tree, filename='<ast>', mode='exec'), ns)
    return ns.get('_result', None)

In [22]:
run("""
a=1
b=2
a+b
""")

3

In [23]:
# this following function will essentially show the python code that will run on the computer, prompting them to run it or not. 

# in the future, you could have a other failsafe methods and also restrict the globals within the python enviroment preventing any unwanted or malicious code. 
def python(code:str):
    "Return result of executing `code` using python. If execution not permitted, returns `#FAIL#`"
    go = input(f'Proceed with execution?\n```\n{code}\n```\n')
    if go.lower()!='y': return '#FAIL#'
    return run(code)

In [24]:
c = askgpt("What is 12 factorial?",
           system = "Use python for any required computations.",
           functions=[schema(python)])


In [25]:
call_func(c)

In [19]:
c

<OpenAIObject chat.completion id=chatcmpl-86UjCbmlpCuJcjIcNIp3YnzLzSiWj at 0x110817d70> JSON: {
  "id": "chatcmpl-86UjCbmlpCuJcjIcNIp3YnzLzSiWj",
  "object": "chat.completion",
  "created": 1696558778,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "python",
          "arguments": "{\n  \"code\": \"import os\\n\\ncurrent_directory = os.getcwd()\\nfile_list = os.listdir()\\n\\n(current_directory, file_list)\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 78,
    "completion_tokens": 38,
    "total_tokens": 116
  }
}